In [48]:
#!/usr/bin/env python3
import argparse
import asyncio
import os
import random
import sys
from sys import platform

import keras
import music21
import numpy as np
import pygame
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Dense, Embedding, TimeDistributed, Dropout, Flatten
from keras.models import Sequential
from loguru import logger
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from midi2audio import FluidSynth
from keras.utils import to_categorical


In [49]:
def setup_logging(level="DEBUG", show_module=False):
    """
    Setups better log format for loguru
    """
    logger.remove(0)    # Remove the default logger
    log_level = level
    log_fmt = u"<green>["
    log_fmt += u"{file:10.10}…:{line:<3} | " if show_module else ""
    log_fmt += u"{time:HH:mm:ss.SSS}]</green> <level>{level: <8}</level> | <level>{message}</level>"
    logger.add(sys.stderr, level=log_level, format=log_fmt, colorize=True, backtrace=True, diagnose=True)



def play_midi(midi_file):

    #Play MIDI
    sf2_path = '/usr/share/soundfonts/freepats-general-midi.sf2'  # path to sound font file
    FluidSynth(sound_font=sf2_path).play_midi(midi_file)


In [50]:
sequence_length = 32
batch_size = 32
epochs = 1000

# Load the musical data using Music21
corpi = [c for c in random.choices(music21.corpus.getComposer("bach"), k=2)]

X = []
y = []
# for piece in corpus:
for corpus in corpi:
    logger.info(f'Parsing {corpus}')
    parsed = music21.converter.parse(corpus)
    elements = parsed.flat.notes
    # Preprocess the data
    for i in tqdm(range(sequence_length, len(elements))):
        current_seq = []
        for j in range(sequence_length):
            current_seq.append([elements[i-j].pitch.pitchClass, elements[i-j].pitch.octave, int(elements[i-j].duration.quarterLength*2)])
        X.append(current_seq)
        y.append([elements[i].pitch.pitchClass, elements[i].pitch.octave, int(elements[i].duration.quarterLength*2)])

# Convert the data to numpy arrays
X = np.array(X)
y = np.array(y)

pitch_classes_count = max(len(np.unique(X[:, :, 0])), len(np.unique(y[:, 0]))) + 1
octaves_count = max(max(np.unique(X[:, :, 1])), max(np.unique(y[:, 1]))) + 1
durations_count = max(max(np.unique(X[:, :, 2])), max(np.unique(y[:, 2]))) + 1
num_classes = durations_count + octaves_count + pitch_classes_count

pitch_classes_encoded = to_categorical(y[:, 0], num_classes=pitch_classes_count)
octaves_encoded = to_categorical(y[:, 1], num_classes=octaves_count)
durations_encoded = to_categorical(y[:, 2], num_classes=durations_count)
y = np.concatenate((pitch_classes_encoded, octaves_encoded, durations_encoded), axis=1)

pitch_classes_encoded = to_categorical(X[:, :, 0], num_classes=pitch_classes_count)
octaves_encoded = to_categorical(X[:, :, 1], num_classes=octaves_count)
durations_encoded = to_categorical(X[:, :, 2], num_classes=durations_count)
X = np.concatenate((pitch_classes_encoded, octaves_encoded, durations_encoded), axis=2)

n_samples = (len(X) // batch_size) * batch_size
X = X[:n_samples]
y = y[:n_samples]

2023-02-25 16:38:08.362 | INFO     | __main__:<module>:12 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/bach/bwv117.4.mxl
100%|██████████| 173/173 [00:00<00:00, 12646.66it/s]
2023-02-25 16:38:08.448 | INFO     | __main__:<module>:12 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/bach/bwv283.mxl
100%|██████████| 288/288 [00:00<00:00, 12365.87it/s]


In [51]:
# Define the model architecture
model = Sequential()

model.add(LSTM(units=100, activation='tanh', return_sequences=True, input_shape=(sequence_length, num_classes)))
# model.add(Dropout(0.2))
# model.add(LSTM(units = 100, activation='tanh', return_sequences = True))
# model.add(Dropout(0.2))
model.add(LSTM(units = 100))
# model.add(Dropout(0.2))
# model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
# model.add(LSTM(4, input_shape=(sequence_length, num_classes)))
# model.add(Dense(num_classes, activation='softmax'))
# model.add(TimeDistributed(Dense(1)))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=epochs)

# Evaluate the model
# score = model.evaluate(X_val, y_val, batch_size=batch_size)
# print("Validation Loss:", score[0])
# print("Validation Accuracy:", score[1])


Epoch 1/1000
14/14 [==============================] - 2s 13ms/step - loss: 8.7302 - categorical_accuracy: 0.0134
Epoch 2/1000
14/14 [==============================] - 0s 14ms/step - loss: 8.6539 - categorical_accuracy: 0.0000e+00
Epoch 3/1000
14/14 [==============================] - 0s 13ms/step - loss: 9.2948 - categorical_accuracy: 0.0000e+00
Epoch 4/1000
14/14 [==============================] - 0s 13ms/step - loss: 9.4075 - categorical_accuracy: 0.0000e+00
Epoch 5/1000
14/14 [==============================] - 0s 13ms/step - loss: 9.4026 - categorical_accuracy: 0.0000e+00
Epoch 6/1000
14/14 [==============================] - 0s 13ms/step - loss: 9.4800 - categorical_accuracy: 0.0000e+00
Epoch 7/1000
14/14 [==============================] - 0s 13ms/step - loss: 9.5576 - categorical_accuracy: 0.0000e+00
Epoch 8/1000
14/14 [==============================] - 0s 13ms/step - loss: 9.6071 - categorical_accuracy: 0.0000e+00
Epoch 9/1000
14/14 [==============================] - 0s 13ms/step -

In [52]:

# Generate music
start_note = np.zeros((1, sequence_length, num_classes))
pitch_classes_encoded = to_categorical(elements[0].pitch.pitchClass, num_classes=pitch_classes_count)
octaves_encoded = to_categorical(elements[0].pitch.octave, num_classes=octaves_count)
durations_encoded = to_categorical(int(elements[0].duration.quarterLength*2), num_classes=durations_count)
start_note[0, -1] = np.concatenate((pitch_classes_encoded, octaves_encoded, durations_encoded), axis=0)
generated_notes = []
notes = start_note
generated_stream = music21.stream.Stream()
for i in range(100):
    prediction = model.predict(notes)
    note = prediction[0]
    pitch_class = np.argmax(note[:pitch_classes_count])
    octave = np.argmax(note[pitch_classes_count: pitch_classes_count + octaves_count])
    duration = np.argmax(note[pitch_classes_count + octaves_count:])
    
    pitch = music21.pitch.Pitch(pitch_class, octave=octave)
    duration = music21.duration.Duration(quarterLength=duration/2)
    note = music21.note.Note(pitch=pitch, duration=duration)
    generated_stream.append(note)
    
    pitch_classes_encoded = to_categorical(note.pitch.pitchClass, num_classes=pitch_classes_count)
    octaves_encoded = to_categorical(note.pitch.octave, num_classes=octaves_count)
    durations_encoded = to_categorical(int(note.duration.quarterLength*2) , num_classes=durations_count)
    note = np.concatenate((pitch_classes_encoded, octaves_encoded, durations_encoded), axis=0)
    for j in range(sequence_length -1):
        for k in range(num_classes):
            notes[:, j, k] = notes[:, j+1, k]
        notes[:, j] = notes[:, j+1]
    notes[:, -1] = note

1/1 [==============================] - 0s 14ms/step


In [53]:

# Convert the generated notes to a Music21 Stream
for note in generated_notes:
    generated_stream.append(note)
    
# Write the generated stream to a MIDI file
generated_stream.write('midi', fp='generated_music.mid')

play_midi('generated_music.mid')

ALSA lib pcm_dsnoop.c:566:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:999:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm_

FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

